# Parameter Tuning - SARIMAX

S + ARIMA + X
* S: Seasonaly
* ARIMA: AutoRegressive Integrated Moving Average
* X: Exogenous regressors

## Libraries and Data

In [ ]:
# Libraries
!pip install pmdarima

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
import pmdarima as pm
from pmdarima import model_selection

In [4]:
# Data
df = pd.read_csv('../Data/nyc-data.csv', index_col=0, parse_dates=True)
df

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077
...,...,...,...,...,...,...
2020-12-27,685.915026,0,0,0,2.89,38.674
2020-12-28,998.051170,0,0,0,8.83,166.712
2020-12-29,847.123399,0,0,0,3.48,161.865


In [5]:
# Rename variable
df = df.rename(columns={'Demand': 'y'})
df.head(1)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305


In [6]:
# Extract regressors
X = df.iloc[:, 1:]
X.head(1)

,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,
2015-01-01,0,0,0,3.68,41.305


## AutoRegressive components

Past values, the lags, contain information that help predict future values.

How many lags should we use?  Use parameter tuning to find out.

## Integrated components

How many times we need to do differencing to achieve stationarity.

Stationarity
* Mean, variance, and covariance are not time dependent
* A stationary time-series has a clearly defined pattern
* Use the Dickey-Fuller test to test for stationarity.  If p < 0.05, then stationary.

Differencing
* Subtracting of consecutive observations
* A way to make time-series data stationary
* A check we need to do to make SARIMAX work correctly

### Stationarity

In [7]:
# Test
from statsmodels.tsa.stattools import adfuller

In [12]:
pvalue = adfuller(x=df.y)[1]

# condition to read test
if pvalue < 0.05:
  print(f'Time-series is stationary. P-value: {pvalue:.2f}')
else:
  print(f'Time-series is NOT stationary. P-value: {pvalue:.2f}')

Time-series is NOT stationary. P-value: 0.38


In [13]:
# Differencing
pvalue = adfuller(x=df.y.diff().dropna())[1]

# condition to read test
if pvalue < 0.05:
  print(f'Time-series is stationary. P-value: {pvalue:.2f}')
else:
  print(f'Time-series is NOT stationary. P-value: {pvalue:.2f}')

Time-series is stationary. P-value: 0.00


## Moving Average components

Use lagged error to help predict the future values, especially if there is a pattern in the error.

Use parameter tuning to find it.

## Optimization Factors

3 factors to optimize ARIMA/ARIMAX (p, d, q)
* p: order of the autoregressive
  * Number of time series lags used
* d: degree of first differencing involved
  * Number of differences to make time series stationary
* q: order of the moving average part
  * Number of forecasting errors lags used

All of these are non-negative integers.

6 factors to optimize SARIMA, SARIMAX
* Seasonal P, D, Q
* Non-seasonal p, d, q


## SARIMAX model

Only allows one kind of seasonality, as indicated by the final value in the `seasonal_order` iterable.
* hourly: 24
* daily: 7
* weekly: 52
* monthly: 12
* quarterly: 4

In [17]:
# Model
model = pm.ARIMA(order=(1, 1, 1),
                 seasonal_order=(1, 1, 1, 7),
                 X=X,
                 suppress_warnings=True,  # Can get lots of warnings with ARMIA
                 force_stationarity=False)

In [19]:
# Cross validation
cv = model_selection.RollingForecastCV(h=31,
                                       step=16,  # Same as Prophet model
                                       initial=df.shape[0] - 180)  # Same as Prophet model
cv_score = model_selection.cross_val_score(model,
                                           y=df.y,
                                           scoring='mean_squared_error',
                                           cv=cv,
                                           verbose=2,
                                           error_score=10000000000000)  # Just a big number

[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=3 ..........................................................
[CV] fold=4 ..........................................................
[CV] fold=5 ..........................................................
[CV] fold=6 ..........................................................
[CV] fold=7 ..........................................................
[CV] fold=8 ..........................................................
[CV] fold=9 ..........................................................


In [20]:
# CV performance
error = np.sqrt(np.average(cv_score))
error

59.95521221161768

## Parameter Tuning

For SARIMAX, start with lower values, then build from there.

In [23]:
# Grid
param_grid = {'p': [0, 1],
              'd': [1],  # Usually yields less errors
              'q': [0, 1],
              'P': [0, 1],
              'D': [0, 1],
              'Q': [0, 1]}
grid = ParameterGrid(param_grid)
len(list(grid))

32

In [24]:
# Parameter Tuning
rmse = []
i = 1

# Parameter loop
for params in grid:
  print(f'{i} / {len(list(grid))}')
  # Build model
  model = pm.ARIMA(order=(params['p'], params['d'], params['q']),
                 seasonal_order=(params['P'], params['D'], params['Q'], 7),
                 X=X,
                 suppress_warnings=True,
                 force_stationarity=False)

  # Cross-validation
  cv = model_selection.RollingForecastCV(h=31,
                                         step=16,
                                         initial=df.shape[0] - 180)
  cv_score = model_selection.cross_val_score(model,
                                             y=df.y,
                                             scoring='mean_squared_error',
                                             cv=cv,
                                             verbose=2,
                                             error_score=10000000000000)  # Just a big number

  # Error
  error = np.sqrt(np.average(cv_score))
  rmse.append(error)

  i += 1

1 / 32
[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=3 ..........................................................
[CV] fold=4 ..........................................................
[CV] fold=5 ..........................................................
[CV] fold=6 ..........................................................
[CV] fold=7 ..........................................................
[CV] fold=8 ..........................................................
[CV] fold=9 ..........................................................
2 / 32
[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=3 ...................................................

In [26]:
# Check the results
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

,D,P,Q,d,p,q,rmse
0,0,0,0,1,0,0,109.196397
1,0,0,0,1,0,1,86.990889
2,0,0,0,1,1,0,88.724335
3,0,0,0,1,1,1,85.350878
4,0,0,1,1,0,0,111.422774
5,0,0,1,1,0,1,84.774305
6,0,0,1,1,1,0,91.853986
7,0,0,1,1,1,1,83.556196
8,0,1,0,1,0,0,134.711122
9,0,1,0,1,0,1,91.895286


In [28]:
# Export best parameters
best_params = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()
best_params.to_csv('../Forecasting-Product/best-params-SARIMAX.csv')